In [ ]:
import pickle
import numpy as np
import torch
from tqdm import tqdm
import numqi
import matplotlib.pyplot as plt

In [ ]:
# Random generate a d^-1-dimensional real vector in Euclidean space
def random_vector(d):
    x = np.random.normal(size=d*d-1)
    x = x / np.linalg.norm(x)
    return x

def save_data(xdata, ydata, filepath='data_boundary.pkl'):
    with open(filepath, 'wb') as fid:
        pickle.dump(dict(xdata=xdata, ydata=ydata), fid)

def load_data(filepath='data_boundary.pkl'):
    with open(filepath, 'rb') as fid:
        tmp0 = pickle.load(fid)
        ret = tmp0['xdata'], tmp0['ydata']
    return ret


In [ ]:
dim = 3
num_sample = 10000
batch_size = 64

In [ ]:
# Training data
vec_list = [random_vector(dim) for _ in range(num_sample//2)]
xdata = []
ydata = []
for vec in tqdm(vec_list):
    rho = numqi.gellmann.gellmann_basis_to_dm(vec)
    beta_dm_l, beta_dm_u = numqi.entangle.get_density_matrix_boundary(rho)
    xdata.append(vec)
    ydata.append(beta_dm_u)
    xdata.append(-vec)
    ydata.append(-beta_dm_l)
xdata = np.array(xdata)
ydata = np.array(ydata)
y_mean = np.mean(ydata)
print(f'min: {np.min(ydata)}, max: {np.max(ydata)}, mean: {y_mean}, variance: {np.var(ydata)}')
save_data(xdata, ydata)
    

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        tmp0 = [dim**2-1, 128, 128, 256, 256, 128, 128, 1]
        self.fc_list = torch.nn.ModuleList([torch.nn.Linear(tmp0[x], tmp0[x+1]) for x in range(len(tmp0)-1)])
        self.bn_list = torch.nn.ModuleList([torch.nn.BatchNorm1d(tmp0[x+1]) for x in range(len(tmp0)-1)])

    def forward(self, x):
        for ind0 in range(len(self.fc_list)):
            y = self.fc_list[ind0](x)
            y = self.bn_list[ind0](y)
            if ind0<(len(self.fc_list)-1):
                y = torch.nn.functional.leaky_relu(y)
            if x.shape[-1]==y.shape[-1]:
                x = y + x
            else:
                x = y
        x = torch.nn.functional.sigmoid(x) + torch.tensor(y_mean, dtype=torch.float32)
        return x

In [ ]:
xdata,ydata = load_data()

tmp0 = torch.tensor(xdata, dtype=torch.float32)
tmp1 = torch.tensor(ydata, dtype=torch.float32)
dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(tmp0, tmp1), batch_size=batch_size, shuffle=True)

In [ ]:
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_history = []
for epoch in range(100):
    model.train()
    for x, y in dataloader:
        optimizer.zero_grad()
        y_pred = model(x)
        loss = torch.nn.functional.mse_loss(y_pred, y)
        loss.backward()
        optimizer.step()
        loss_history.append(loss.item())
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}: {loss.item()}')

In [ ]:
fig,ax = plt.subplots()
ax.plot(np.arange(len(loss_history)), loss_history, '.')
ax.set_xlabel('iteration')
ax.set_ylabel('loss')
# log scale
ax.set_yscale('log')
ax.grid()
fig.tight_layout()

In [ ]:
i=1
j=6
GellMann = numqi.gellmann.all_gellmann_matrix(dim, with_I=False)
op0 = GellMann[i]
op1 = GellMann[j]
print(op0)
print(op1)

In [ ]:
theta_list = np.linspace(0, 2*np.pi, 500)
direction = np.stack([np.cos(theta_list), np.sin(theta_list)], axis=1)

_, hf_plane = numqi.entangle.get_density_matrix_plane(op0, op1)
beta_dm_cs = np.array([numqi.entangle.get_density_matrix_boundary(hf_plane(x))[1] for x in theta_list])
nr_dm_cs = beta_dm_cs.reshape(-1,1) * direction

vec_model = np.zeros((len(theta_list), dim**2-1))
vec_model[:,i] = np.cos(theta_list)
vec_model[:,j] = np.sin(theta_list) 
model.eval()
with torch.no_grad():
    beta_dm_model = model(torch.tensor(vec_model, dtype=torch.float32)).numpy()

nr_dm_model_cs = beta_dm_model.reshape(-1,1) * direction

fig, ax = plt.subplots()
ax.plot(nr_dm_cs[:,0], nr_dm_cs[:,1], '-')
ax.plot(nr_dm_model_cs[:,0], nr_dm_model_cs[:,1], '--')
ax.set_xlabel(r'$O_0$')
ax.set_ylabel(r'$O_1$')